In [1]:
#mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# move into project directory
repo_name = "UMass-CS670-Project"
%cd /content/drive/MyDrive/UMass-Courses/Semester-2/CS670/$repo_name
!ls

/content/drive/MyDrive/UMass-Courses/Semester-2/CS670/UMass-CS670-Project
common	datautils    get_dataset.sh	      Index.ipynb  models     SINet.ipynb
data	experiments  github-repo-clone.ipynb  index.py	   README.md


In [ ]:
# set up environment
# comment out if not required
'''
!pip3 install torch torchvision torchaudio
!pip install matplotlib numpy pandas pyyaml opencv-python
'''

'\n!pip3 install torch torchvision torchaudio\n!pip install matplotlib numpy pandas pyyaml opencv-python\n'

In [14]:
# this cell is for downloading data.
# as of yet data is not hosted and is available in the private data folder
# uncomment the lines below  to download the data

import os

!pip install gdown

data_dir="data"

if not(os.path.exists("data")) or not(os.path.exists("./data/COD10K-v3")):
    !gdown https://drive.google.com/uc?id=18oBjWeuw5qAq4HG_jZdjUHas4APy-KJE&export=download
    if not(os.path.exists("data")):
        os.mkdir("data")
    #echo 'Downloaded data! Unzipping to data folder'
    !unzip -qq -d ./$data_dir ./COD10K-v3.zip
    os.rename("./COD10K-v3.zip", "./data/COD10K-v3.zip")
else:
    print("\nThe data directory exists!")

Downloading...
From (original): https://drive.google.com/uc?id=18oBjWeuw5qAq4HG_jZdjUHas4APy-KJE
From (redirected): https://drive.google.com/uc?id=18oBjWeuw5qAq4HG_jZdjUHas4APy-KJE&confirm=t&uuid=a9b72044-8371-4cd7-8ef5-8f9e0fd8bbfc
To: /content/drive/MyDrive/UMass-Courses/Semester-2/CS670/UMass-CS670-Project/COD10K-v3.zip
100% 995M/995M [00:07<00:00, 134MB/s]


In [15]:
# setup some imports
from datautils.datareader import read_data
from datautils.dataset import COD10KDataset
from torch.utils.data import DataLoader
import random
import numpy as np
import torch
import argparse
from tqdm import tqdm
from torchvision.io import read_image
from torchvision.utils import save_image
from torchvision.transforms.functional import to_pil_image
import json

from experiments.style_transfer import style_transfer
from common.visualizer import layer_visualizer

In [4]:
seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [13]:
def run_style_transfer_pipeline(args, texture_name, style_weight, last_batch_run = -1):
    pos_data_paths = read_data('Train')

    dataset = COD10KDataset(pos_data_paths)
    dataloader = DataLoader(dataset, batch_size = args.batch_size, shuffle = False)

    for i_batch, batch in enumerate(dataloader):
        if i_batch > last_batch_run:
            style_img = read_image(f'./data/Textures/{texture_name}.jpg')
            new_img = style_transfer(
                batch['img'],
                style_img,
                [0, 2, 5, 14, 23],
                21,
                1e-4,
                [style_weight]*5,
                1e-5,
                args)
            img_name = batch['img_name']
            img_name = img_name.replace(".jpg", "")
            img_name = f"{img_name}-Texture-{texture_name}.jpg"
            save_image(new_img, f'./data/COD10K-v3/Train/Styled-Image/butterfly-image-Texture-{texture_name}.jpg')
            last_run = {
                "last_batch": i_batch
            }
            with open("./last_run_info.json", "w") as fp:
                json.dump(last_run, fp)

    '''
    style_img = read_image(f'./data/Textures/{texture_name}.jpg')
    content_img = read_image('./data/Mini-Set/butterfly-image.jpg')
    #content_img = batch['img']
    print('b4 trans', content_img.shape)
    new_img = style_transfer(content_img,
            style_img,
            [0, 2, 5, 14, 23],
            21,
            1e-4,
            [style_weight]*5,
            1e-5,
            args)
    #pi = to_pil_image(pi)
    #plt.axis("off")
    #plt.imshow(pi)
    #plt.show()
    save_image(new_img, f'./data/COD10K-v3/Train/Styled-Image/butterfly-image-Texture-{texture_name}.jpg')
    #print('after trans', new_img.size())
    '''

In [17]:

param_dict = {
    'moss': {
        'epochs': 100,
        'style_weights': 0.1
    },
    'snow': {
        'epochs': 200,
        'style_weights': 1.5
    },
    'wet-sand': {
        'epochs': 200,
        'style_weights': 3.5
    },
    'sand-with-moss': {
        'epochs': 100,
        'style_weights': 1.5
    },
    'rain': {
        'epochs': 100,
        'style_weights': 3.5
    },
    'wood': {
        'epochs': 100,
        'style_weights': 0.5
    },
    'grass': {
        'epochs': 100,
        'style_weights': 0.1
    },
    'foliage-texture': {
        'epochs': 100,
        'style_weights': 0.1
    },
    'blue-coral': {
        'epochs': 100,
        'style_weights': 1.1
    }
}

with open("./last_run_info.json") as fp:
    last_run = json.load(fp)["last_batch"]

for texture in param_dict.keys():
    parser = argparse.ArgumentParser()
    parser.add_argument('--batch_size', type = int, default = 1)
    parser.add_argument('--model_name', type=str, default='vgg')
    parser.add_argument('--lr', type=float, default=0.05)
    parser.add_argument('--max_iter', type=int, default=param_dict[texture]['epochs'])
    args = parser.parse_args(args=[])

    #style_img = read_image(f'./data/Textures/{texture}.jpg')
    #print(img.size())
    #layer_visualizer(img, args)

    run_style_transfer_pipeline(args, texture, param_dict[texture]['style_weights'], last_run)

RuntimeError: [Errno 2] No such file or directory: 'data/COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-23-Cat-1497.jpg'